In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
 !unzip /kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip #압축풀기

In [ ]:
!unzip /kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip

In [ ]:
from PIL import Image #파이썬 이미지 보기
Image.open("test/10278.jpg")

In [ ]:
import glob #경로를 알아내는 라이브러리
glob.glob("train/*")

In [ ]:
train = pd.DataFrame({'path':glob.glob("train/*")})

In [ ]:
test = pd.DataFrame({'path':glob.glob("test/*")})

In [ ]:
train["target"] = train["path"].apply(lambda x:x.split("/")[1].split('.')[0])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
idg = ImageDataGenerator()
train_generator = idg.flow_from_dataframe(train,x_col="path",y_col='target',target_size=(300,300),batch_size=32)#ram떄문에 끊어서 학습
test_generator = idg.flow_from_dataframe(test,x_col='path',y_col=None, target_size=(300,300), batch_size=32,class_mode=None, shuffle=False)  #정답값이 없기 떄문에 이렇게 한다.

In [ ]:
test_generator.filenames

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
from keras.models import Sequential
from keras.layers import *
from keras.applications.inception_v3 import InceptionV3
from efficientnet.tfkeras import EfficientNetB0
from keras.optimizers import SGD
model = Sequential() 
model.add(EfficientNetB1(weights="imagenet",include_top=False,pooling="avg")) #weight : 다른 곳에서 이미 학습했던 weight를 먼저 가지고 옴(전이 학습) , pooling="avg"는 flatten을 대신해서 정보 손실을 감소
model.add(Dense(2,activation="softmax"))
model.compile(metrics=["acc"],loss="categorical_crossentropy",optimizer=SGD(lr=0.01,momentum=0.9, nesterov=True))# lr=learning rate, momentom,nesterov = 경사하강에서 방향을 너무 확바꾸지 않도록 합벡터를 통해


In [ ]:
# model =  Sequential()
# model.add(Conv2D(32,(3,3), activation="relu",input_shape=(100,100,3))) #맨앞의 숫자는 몇개의 특징을 추출할 것인지
# model.add(Conv2D(64,(3,3), activation="relu"))
# model.add(MaxPooling2D())
# model.add(Conv2D(128,(3,3), activation="relu"))
# model.add(MaxPooling2D())
# model.add(Dropout(0.1))
# model.add(Flatten())
# model.add(Dense(2, activation="softmax"))
# model.compile(metrics=["acc"],loss="categorical_crossentropy",optimizer="adam") #분류 문제임이 당연하기 때문에 sparse_ 없어도 됨

In [ ]:
model.fit(train_generator, epochs=1)

result = model.predict(test_generator)

In [ ]:
sub = pd.read_csv("/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv")
sub['id'] = test_generator.filenames
sub['id'] = sub['id'].apply(lambda x:x.split("/")[1].split(".")[0])
sub["label"] = result[:,1],clip(0.005, 0.995) #log_loss를 사용하기 떄문에 값이 튀는(0,1처럼) 것들을 잘라줌 

In [ ]:
sub

In [ ]:
sub.to_csv("cat_dog_sub.csv",index=0)